A bit levelled up for the video classification problems?!

Don't worry, we got you!

Insatlling library for plotting graphs

In [ ]:
!pip install plot-metric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Importing required libraries (majorly used keras and tensorflow)

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
import sklearn
import matplotlib.pyplot as plt

Of course, picking the modules we gotta play with.

In [ ]:
from glob import glob
from tqdm import tqdm
from keras.optimizers import Adam
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dropout, Dense, Flatten, ConvLSTM2D,Conv2D,MaxPool2D,GlobalMaxPool2D,GlobalAveragePooling2D,BatchNormalization,Input,TimeDistributed,LSTM,Reshape,GlobalAveragePooling2D
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report,accuracy_score,precision_score,recall_score,f1_score,roc_auc_score,roc_curve,confusion_matrix
from keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers.experimental import preprocessing
from plot_metric.functions import BinaryClassification
from mlxtend.plotting import plot_confusion_matrix
from sklearn.utils import shuffle
from tabulate import tabulate

Easy part gone. Lets get into what is required in problem.

Okay, has there been any algorithm or model made for video classification???? No, I guess. But I think we heard of something known as CNN which can be applied on images.

Wait, videos are composed of multiple images or frames, so it can be done by convertig videos into frames keeping in mind to maintain sequence of the frame.

In [ ]:
data_directory = "/content/drive/MyDrive/Violence Detection Dataset/Merged"
img_height,img_width = 112,112
seq_len = 10
classes = ["NonViolence", "Violence"]

In [ ]:
fps=0
frame_count=0
duration=0

Extracting 10 frames from each video to create a dataset

In [ ]:
def frames_extraction(video_path):
    frames_list = []

    vidObj = cv2.VideoCapture(video_path)
    global fps
    fps = vidObj.get(cv2.CAP_PROP_FPS)
    global frame_count
    frame_count+= int(vidObj.get(cv2.CAP_PROP_FRAME_COUNT))
    global duration
    duration += frame_count/fps
    count = 1
    total_frames = vidObj.get(cv2.CAP_PROP_FRAME_COUNT)
    frames_step = total_frames//seq_len
    for i in range(seq_len):
        vidObj.set(1,i*frames_step)
        success,image = vidObj.read()
        if success:
            image = cv2.resize(image, (img_height, img_width))
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            frames_list.append(image)
            count += 1
        else:
            print("Defected frame")
            break

    return frames_list

In [ ]:
def create_data(input_dir):
    X = []
    Y = []

    classes_list = os.listdir(input_dir)
    for c in classes_list:
        print(c)
        files_list = os.listdir(os.path.join(input_dir, c))
        for f in tqdm(files_list):
            frames = frames_extraction(os.path.join(os.path.join(input_dir, c), f))
            if len(frames) == seq_len:
                X.append(frames)
                y = [0]*len(classes)
                y[classes.index(c)] = 1
                Y.append(y)
    print("Frames per second:",fps)
    print("Average number of frames in a video: ",frame_count/4000)
    X = np.asarray(X)
    Y = np.asarray(Y)
    return X, Y

X, Y = create_data(data_directory)

FileNotFoundError: ignored

Saving the created data so that you need not to create it again and again.

In [ ]:
np.save('/content/drive/MyDrive/Violence Detection Dataset/X_112_112_10_cnn',X)
np.save('/content/drive/MyDrive/Violence Detection Dataset/Y_112_112_10_cnn',Y)

Loading data

In [ ]:
X=np.load('/content/drive/MyDrive/Violence Detection Dataset/X_112_112_10_cnn.npy')
Y=np.load('/content/drive/MyDrive/Violence Detection Dataset/Y_112_112_10_cnn.npy')

Shuffling data

In [ ]:
X,y = shuffle(X,Y, random_state=42)
print("The shape of input is : ",np.shape(X))
print("The shape of target is : ", np.shape(y))

We gonna do 5 folds for the model and then we move forward with model.

Did you miss anything?

Preprocessing?

No, we did it very right now go back to the block where we were trying to generate the frames. Thats the point where we preprocessed frames from BGR to RGB.

In [ ]:
SIZE = (112,112)
CHANNELS = 3
NBFRAME = 10
INSHAPE=(NBFRAME,) + SIZE+ (CHANNELS,)

splits=5
skf = StratifiedKFold(n_splits=splits, shuffle=False)
y=np.argmax(y,axis=1)

Now, a moderate difficult state, the state where we try to create model and doing layers and stuff.

Since we are dumping our final model on raspberry pi, be careful with the number of parameters( not try to exceed 1.2-1.3 million.)

So below is the CNN model, so how are we going to do it sequentially( frames to be passes in sequence) ?

Lets see next block, CNN is bit easy.

In [ ]:
def build_convnet(shape=(112, 112, 3)):
    momentum = .9
    model = Sequential([preprocessing.Rescaling(1.0 / 255),
                              preprocessing.RandomFlip("horizontal"),
                              preprocessing.RandomFlip("vertical"),
                              preprocessing.RandomRotation(0.1),
                              preprocessing.RandomZoom(0.1)])

    model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))

    model.add(MaxPool2D())

    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))

    model.add(MaxPool2D())

    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))

    model.add(MaxPool2D())

    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))

    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))

    model.add(GlobalAveragePooling2D())
    return model

Here is the difficult part where we do sequence passing of frames using time distributed layer. And our modelling part is complete.

In [ ]:
def convnet_lstm_model(shape=(NBFRAME,112,112,3), nbout=2):
    convnet = build_convnet(shape[1:])
    model = Sequential()
    model.add(TimeDistributed(convnet, input_shape=shape))
    model.add(LSTM(64))
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nbout, activation='softmax'))
    return model

Checking out number of parameters to see if it will fit in raspberry pi.

In [ ]:
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 15, mode = 'min', restore_best_weights = False)

model_convnet = convnet_lstm_model(INSHAPE)
model_convnet.summary()

Visualizing our model in a flowchart.

In [ ]:
plot_model(model_convnet,show_shapes=True,expand_nested=True)

Training the model using Adam optimizer on our data to verify training loss and validation loss.

In [ ]:
training_loss=[]
training_accu=[]
valid_accu=[]
valid_loss=[]
pred_targets=[]
Y_test_pf=[]
fold=1
for train_index, test_index in skf.split(X, y):
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = y[train_index], y[test_index]
  y_train=np.asarray(pd.get_dummies(y_train))
  y_test=np.asarray(pd.get_dummies(y_test))

  model_convnet = convnet_lstm_model(INSHAPE)
  optimizer = keras.optimizers.Adam(1e-4)
  model_convnet.compile(optimizer,'categorical_crossentropy',metrics=['acc'])
  print("------------------------------------------------------------------------------------------------------------------------------------------")
  print("------------------------------------------------------------------------------------------------------------------------------------------")
  print("Fold {}".format(fold))
  history = model_convnet.fit(x=X_train, y=y_train, epochs=50, batch_size = 8,validation_split=0.125,callbacks = [early_stopping_callback])
  print()
  model_evaluation_history = model_convnet.evaluate(X_test,y_test)
  pred_targets.append(model_convnet.predict(X_test))

  training_accu.append(history.history['acc'])
  training_loss.append(history.history['loss'])
  valid_accu.append(history.history['val_acc'])
  valid_loss.append(history.history['val_loss'])
  Y_test_pf.append(y_test)
  fold +=1

Looks pretty good training been done on dataset. Let's check results!!

#**Convnet + LSTM :**



##**Results:**

Visualizing loss and accuracy graphs.

In [ ]:
plt.figure(figsize=(12,9))
plt.plot(history.history['loss'],'g')
plt.plot(history.history['val_loss'],'r')
plt.title('Training and Validation Loss')
plt.legend(["Training","Validation"])
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.show()

plt.figure(figsize=(12,9))
plt.plot(history.history['acc'],'g')
plt.plot(history.history['val_acc'],'r')
plt.title('Training and Validation accuracy')
plt.legend(["Training","Validation"],loc="lower right")
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.show()

Various metrics such as accuracy, recall, precision, f1-score, roc and etc.

In [ ]:
avg_accuracy=0
avg_recall=0
avg_precision=0
avg_f1=0
avg_fpr=0
avg_fnr=0
table=[]
for i in range(splits):
  print("------------------------------------------------------------------------------------------------------------------------------------------")
  print("For Fold {}".format(i+1))

  y_pred=pred_targets[i]
  y_prob=y_pred
  y_pred=np.argmax(y_pred,axis=1)
  Y_test=np.argmax(Y_test_pf[i],axis=1)
  conf_mat=confusion_matrix(Y_test,y_pred)
  table.append(["Fold {}".format(i+1),accuracy_score(Y_test,y_pred),f1_score(Y_test,y_pred),recall_score(Y_test,y_pred),precision_score(Y_test,y_pred),conf_mat[0][1]/(conf_mat[0][1]+conf_mat[0][0]),conf_mat[1][0]/(conf_mat[1][0]+conf_mat[1][1])])
  avg_accuracy+=accuracy_score(Y_test,y_pred)
  avg_recall +=recall_score(Y_test,y_pred)
  avg_precision +=precision_score(Y_test,y_pred)
  avg_f1 += f1_score(Y_test,y_pred)
  avg_fpr += conf_mat[0][1]/(conf_mat[0][1]+conf_mat[0][0])
  avg_fnr += conf_mat[1][0]/(conf_mat[1][0]+conf_mat[1][1])

  print("Correctly predicted out of 800: ",sum(Y_test==y_pred))
  print("Accuracy is                   : ",accuracy_score(Y_test,y_pred))
  print("F1-score is                   : ",f1_score(Y_test,y_pred))
  print("Recall score is               : ", recall_score(Y_test,y_pred))
  print("Precision score is            : ", precision_score(Y_test,y_pred))
  print("False Positive Rate           : ", conf_mat[0][1]/(conf_mat[0][1]+conf_mat[0][0]))
  print("False Negative Rate           : ", conf_mat[1][0]/(conf_mat[1][0]+conf_mat[1][1]))
  print("Area under the curve of ROC is: ",roc_auc_score(Y_test,y_prob[:,1]))
  print()
  print("Report")
  print(classification_report(Y_test,y_pred))
  bc = BinaryClassification(Y_test, y_prob[:,1], labels=["Class 1", "Class 2"])
  plt.figure(figsize=(10,10))
  bc.plot_roc_curve()
  plt.show()
  fig, ax = plot_confusion_matrix(conf_mat,show_absolute=True,show_normed=True,colorbar=True)
  plt.title("Confusion matrix")
  plt.show()

Average ROC curve for  5 folds for CNN+LSTM.

In [ ]:
from sklearn.metrics import auc
tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 101)

plt.figure(figsize=(10,10))

for i in range(splits):
    y_prob=pred_targets[i]
    Y_test=np.argmax(Y_test_pf[i],axis=1)
    aucs.append(roc_auc_score(Y_test,y_prob[:,1]))
    fpr,tpr,_ = roc_curve(Y_test, y_prob[:,1])
    plt.plot(fpr, tpr, alpha=0.25, label=r'Fold %d (AUC = %0.2f)' % (i+1, aucs[-1]))
    interp_tpr = np.interp(mean_fpr, fpr, tpr)
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)


tprs = np.array(tprs)
mean_tprs = tprs.mean(axis=0)
std = tprs.std(axis=0)

tprs_upper = np.minimum(mean_tprs + std, 1)
tprs_lower = np.maximum(mean_tprs - std,0)

mean_auc=auc(mean_fpr, mean_tprs)
std_auc=np.std(aucs)
plt.plot(mean_fpr, mean_tprs, 'b',lw=2,label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc))
plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=0.4,label=r'$\pm$ 1 std. dev.')
plt.title("Average ROC for CNN+LSTM")
plt.plot([0, 1], [0, 1],'r--',label='Random Guess',lw=2)
plt.xlim([-0.01, 1.01])
plt.ylim([-0.01, 1.01])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')

plt.legend(loc="lower right")
plt.show()

Making table for various folds.

In [ ]:
tbl=np.array(table)
table.append(['Average',(avg_accuracy/splits),(avg_f1/splits),(avg_recall/splits),(avg_precision/splits),(avg_fpr/splits),(avg_fnr/splits)])
table.append(['Standard Deviation',np.std(tbl[:,1].astype('float32')),np.std(tbl[:,2].astype('float32')),np.std(tbl[:,3].astype('float32')),np.std(tbl[:,4].astype('float32')),np.std(tbl[:,5].astype('float32')),np.std(tbl[:,6].astype('float32'))])
print(tabulate(table,headers=["","Accuracy","F1-score","Recall","Precison","FPR","FNR"],tablefmt='github'))
print("")